# Les annotations

En java la annotations sont un moyen d'associer des meta-données  aux classes, méthodes, propriétés, paramètres, variables et ... aux annotations. 

Ces méta-données peuvent être utilisés lors d'une traitement du code source, lors de la compilation ou pendant l'exécution. Par exemple pour détecter ou ignorer des erreurs lors de la compilation, générer de la documentation, du code ou des fichiers. 

Il existe des annotations standards [Deprecated](https://docs.oracle.com/en/java/javase/17/docs/api/java.base/java/lang/Deprecated.html), [FunctionalInterface](https://docs.oracle.com/en/java/javase/17/docs/api/java.base/java/lang/FunctionalInterface.html), [Override](https://docs.oracle.com/en/java/javase/17/docs/api/java.base/java/lang/Override.html), [SuppressWarings](https://docs.oracle.com/en/java/javase/17/docs/api/java.base/java/lang/SuppressWarnings.html) et [SafeVarargs](https://docs.oracle.com/en/java/javase/17/docs/api/java.base/java/lang/SafeVarargs.html)

## Définition d'une annotation
Une annotation se déclare dans un fichier qui a le même nom comme une classe ou une interface

In [1]:
public @interface MyAnnotation { }

Elle peut alors être utilisée.

In [2]:
@MyAnnotation
public class MyClass {
    @MyAnnotation
    private int x;
    
    @MyAnnotation
    public void m(@MyAnnotation int y) {
        @MyAnnotation
        String m="test";
    }
}

Si l'on y ajoute des méthodes leurs types (primitifs, référence vers la class Class, annotation, enumération ou un tableau des précédents) et leurs noms définissent respectibement des paramètres. Il est aussi possible de donner des valeur par défaut.  

In [3]:
public @interface MyAnnotation {
    String message();
    int size() default 10;
}

In [4]:
@MyAnnotation(message="hello")
public class MyClass {
}

Si leur attribut obligatoire s'appelle value, il est possible d'omettre son nom lors de l'annotation.

In [5]:
public @interface MyAnnotation {
    String value();
    int size() default 10;
}

In [6]:
@MyAnnotation("hello")
public class MyClass {
}

Une déclaration d'annotation peut être annotée avec [Target](https://docs.oracle.com/en/java/javase/17/docs/api/java.base/java/lang/annotation/Target.html) pour contrôler ce qui peut être annoté :  ANNOTATION_TYPE, CONSTRUCTOR, FIELD, LOCAL_VARIABLE, METHOD, PACKAGE, MODULE, PARAMETER, TYPE, and TYPE_PARAMETER

In [7]:
import java.lang.annotation.Target;
import java.lang.annotation.ElementType;
@Target({ElementType.LOCAL_VARIABLE, ElementType.FIELD})
public @interface MyAnnotation {
    int value() default 0;
}

In [8]:
public class MyClass {    
    @MyAnnotation
    int x;
    public void m() {@MyAnnotation int x;}
}

In [9]:
@MyAnnotation
public class MyClass {    
    int x;
}

CompilationException: 

Par défaut, une annotation ne peut pas être utilisée plusieurs fois. Il faut alors soit définir un wrapper (une annotations qui peut les contenir) 

In [10]:
@Target({ElementType.LOCAL_VARIABLE, ElementType.FIELD})
public @interface MyAnnotations {
    MyAnnotation[] value();
}

In [11]:
public class MyClass {    
    @MyAnnotations({@MyAnnotation(1),@MyAnnotation(2)})
    int x;
}

ou les autoriser à être répétable avec @Repeatable pour éviter la déclaration explicite du tableau.

In [12]:
import java.lang.annotation.Target;
import java.lang.annotation.ElementType;
import java.lang.annotation.Repeatable;

@Repeatable(MyAnnotations.class)
@Target({ElementType.LOCAL_VARIABLE, ElementType.FIELD})
public @interface MyAnnotation {
    int value() default 0;
}

CompilationException: 

In [13]:
public class MyClass {    
    @MyAnnotation(1)
    @MyAnnotation(2)
    int x;
}

Il est aussi possible d'annoter un package dans le fichier package-info.java.

L'annotation Retention permet précision si l'annotation n'est accessible que pendant la compilation (SOURCE), si en plus elle est ajoutée au fichier .class (CLASS) ou si elle est accessible durant l'exécution (RUNTIME).

Inherited permet d'indiquer que l'annotation d'un Type est associé à ses descendants et Documented qu'elle apparait dans la javadoc.

In [14]:
import java.lang.annotation.Documented;
import java.lang.annotation.ElementType;
import java.lang.annotation.Inherited;
import java.lang.annotation.Retention;
import java.lang.annotation.RetentionPolicy;
import java.lang.annotation.Target;

@Documented
@Target(ElementType.METHOD)
@Inherited
@Retention(RetentionPolicy.RUNTIME)
public @interface MyAnnotation {
  public enum Status {IN_PROGRESS, DONE} 
  Status status() default MyAnnotation.Status.DONE;
}

In [15]:
public class MyClass {    
    @MyAnnotation
    public void m1() {System.out.println("Hello1");}
    
    public void m2() {System.out.println("Hello2");}
    
    @MyAnnotation
    public void m3() {System.out.println("Hello3");}
}

Les annotations peuvent être utilisée lors de l'exécution par introspection à condition d'avoir la rétention adaptée. La class Class permet de représenter une classe et d'accéder à la représentation de ses constituants représentés chacun par une classe spécifique (Method, Field, Constructor, ...). Ensuite des actions peuvent être déclenchée de façon programmatique par exemple invoquer une méthode sur une instance, invoquer un constructeur, ...

L'exemple ci-dessus affiche les méthodes de la classe précédent annotées avec @MyAnnotation.

In [16]:
import java.lang.reflect.Method;
MyClass o = new MyClass();
Arrays.stream(o.getClass().getDeclaredMethods())
    .filter(m->m.getAnnotation(MyAnnotation.class)!=null)
    .map(Method::getName)
    .forEach(System.out::println);

m1
m3


Il est aussi possible d'invoquer dynamiquement ces méthodes.

In [17]:
import java.lang.reflect.Method;
MyClass myObject = new MyClass();
Arrays.stream(myObject.getClass().getDeclaredMethods())
    .filter(m->m.getAnnotation(MyAnnotation.class)!=null)
    .forEach(m->{
        try {m.invoke(myObject);} 
        catch (Exception e) {}
        });

Hello1
Hello3


## Annotation processor
